In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

In [2]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


In [9]:
train_data=np.load('training_data.npy')
train_label=np.load('new_train_label.npy')
public_data=np.load('public_data.npy')
public_label=np.load('new_public_label.npy')
private_data=np.load('private_data.npy')
private_label=np.load('new_private_label.npy')

In [10]:

train_data=np.reshape(train_data,(-1,1,48,48))
public_data=np.reshape(public_data,(-1,1,48,48))
private_data=np.reshape(private_data,(-1,1,48,48))


In [11]:
train_data=torch.tensor(train_data)
train_label=torch.tensor(train_label)
public_data = torch.tensor(public_data)

public_label=torch.tensor(public_label)
private_data = torch.tensor(private_data)

private_label=torch.tensor(private_label)

In [5]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image
def reflatten(x):
    N=x.shape[0]
    return x.view(N,8,24,24)

In [6]:
def check_accuracy_part34(test_data,test_label, model):
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        total=int(len(test_data)/64)
        for j in range(total):
            x=test_data[j*64:(j+1)*64]
            y=test_label[j*64:(j+1)*64]
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [7]:
def train_part34(model, optimizer, train_data,train_label,test_data,test_label,epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        print("epoch%i"%e)
        total=int(len(train_data)/64)
        for i in range(total):
            model.train()
            x=train_data[i*64:(i+1)*64]
            y=train_label[i*64:(i+1)*64]
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()
            
#             check_accuracy_part34(test_data,test_label, model)
#             print()
            if i % print_every == 0:
                print('Iteration %d, loss = %.4f' % (i, loss.item()))
                print('train_accuracy')
                check_accuracy_part34(train_data,train_label, model)
                print('validation_accuracy')
                check_accuracy_part34(test_data,test_label, model)
                
                print()
        check_accuracy_part34(test_data,test_label, model)
        print()

In [14]:
import torch.nn.functional as F  # useful stateless functions
# We need to wrap `flatten` function in a module in order to stack it
# in nn.Sequential
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)
class reFlatten(nn.Module):
    def forward(self,x):
        return reflatten(x)

hidden_layer_size = 4000
learning_rate = 1e-4

model = nn.Sequential(
    nn.Conv2d(1,16,kernel_size=(5,5),padding=2),
    nn.ReLU(),
   
    nn.Conv2d(16,8,kernel_size=3,padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    Flatten(),
    nn.Linear(8*24*24, 800),
    nn.ReLU(),
    nn.Linear(800, 400),
    nn.ReLU(),
    nn.Linear(400,100),
    nn.ReLU(),
    nn.Linear(100,50),
    nn.ReLU(),
    nn.Linear(50,7)
)
#model=FiveLayerConvNet(in_channel=1,channel_1=20,channel_2=100,channel_3=30,channel_4=10,num_classes=7)
# # you can use Nesterov momentum in optim.SGD
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                     momentum=0.9, nesterov=True)

train_part34(model,optimizer,train_data,train_label,public_data,public_label,epochs=60)

epoch0
Iteration 0, loss = 1.9594
train_accuracy
Got 5202 / 28672 correct (18.14)
validation_accuracy
Got 619 / 3584 correct (17.27)

Iteration 100, loss = 1.8686
train_accuracy
Got 7227 / 28672 correct (25.21)
validation_accuracy
Got 894 / 3584 correct (24.94)

Iteration 200, loss = 1.8383
train_accuracy
Got 8045 / 28672 correct (28.06)
validation_accuracy
Got 1007 / 3584 correct (28.10)

Iteration 300, loss = 1.7939
train_accuracy
Got 7411 / 28672 correct (25.85)
validation_accuracy
Got 922 / 3584 correct (25.73)

Iteration 400, loss = 1.8016
train_accuracy
Got 8880 / 28672 correct (30.97)
validation_accuracy
Got 1134 / 3584 correct (31.64)

Got 1177 / 3584 correct (32.84)

epoch1
Iteration 0, loss = 1.7139
train_accuracy
Got 9277 / 28672 correct (32.36)
validation_accuracy
Got 1165 / 3584 correct (32.51)

Iteration 100, loss = 1.7852
train_accuracy
Got 8544 / 28672 correct (29.80)
validation_accuracy
Got 1080 / 3584 correct (30.13)

Iteration 200, loss = 1.7762
train_accuracy
Got 98

Got 13514 / 28672 correct (47.13)
validation_accuracy
Got 1526 / 3584 correct (42.58)

Got 1512 / 3584 correct (42.19)

epoch12
Iteration 0, loss = 1.2731
train_accuracy
Got 13495 / 28672 correct (47.07)
validation_accuracy
Got 1502 / 3584 correct (41.91)

Iteration 100, loss = 1.5368
train_accuracy
Got 13129 / 28672 correct (45.79)
validation_accuracy
Got 1526 / 3584 correct (42.58)

Iteration 200, loss = 1.4174
train_accuracy
Got 13676 / 28672 correct (47.70)
validation_accuracy
Got 1519 / 3584 correct (42.38)

Iteration 300, loss = 1.5608
train_accuracy
Got 13311 / 28672 correct (46.43)
validation_accuracy
Got 1480 / 3584 correct (41.29)

Iteration 400, loss = 1.4745
train_accuracy
Got 13899 / 28672 correct (48.48)
validation_accuracy
Got 1522 / 3584 correct (42.47)

Got 1530 / 3584 correct (42.69)

epoch13
Iteration 0, loss = 1.2359
train_accuracy
Got 13804 / 28672 correct (48.14)
validation_accuracy
Got 1512 / 3584 correct (42.19)

Iteration 100, loss = 1.5158
train_accuracy
Got 1

train_accuracy
Got 16971 / 28672 correct (59.19)
validation_accuracy
Got 1601 / 3584 correct (44.67)

Iteration 400, loss = 1.1442
train_accuracy
Got 18325 / 28672 correct (63.91)
validation_accuracy
Got 1639 / 3584 correct (45.73)

Got 1642 / 3584 correct (45.81)

epoch24
Iteration 0, loss = 0.8826
train_accuracy
Got 18068 / 28672 correct (63.02)
validation_accuracy
Got 1638 / 3584 correct (45.70)

Iteration 100, loss = 1.0187
train_accuracy
Got 18266 / 28672 correct (63.71)
validation_accuracy
Got 1679 / 3584 correct (46.85)

Iteration 200, loss = 0.9095
train_accuracy
Got 18090 / 28672 correct (63.09)
validation_accuracy
Got 1595 / 3584 correct (44.50)

Iteration 300, loss = 1.1830
train_accuracy
Got 17323 / 28672 correct (60.42)
validation_accuracy
Got 1623 / 3584 correct (45.28)

Iteration 400, loss = 1.1105
train_accuracy
Got 18782 / 28672 correct (65.51)
validation_accuracy
Got 1670 / 3584 correct (46.60)

Got 1635 / 3584 correct (45.62)

epoch25
Iteration 0, loss = 0.8323
train

Iteration 200, loss = 0.3094
train_accuracy
Got 20502 / 28672 correct (71.51)
validation_accuracy
Got 1479 / 3584 correct (41.27)

Iteration 300, loss = 0.3686
train_accuracy
Got 19605 / 28672 correct (68.38)
validation_accuracy
Got 1574 / 3584 correct (43.92)

Iteration 400, loss = 0.2565
train_accuracy
Got 22203 / 28672 correct (77.44)
validation_accuracy
Got 1585 / 3584 correct (44.22)

Got 1562 / 3584 correct (43.58)

epoch36
Iteration 0, loss = 0.5286
train_accuracy
Got 21487 / 28672 correct (74.94)
validation_accuracy
Got 1540 / 3584 correct (42.97)

Iteration 100, loss = 0.3433
train_accuracy
Got 21703 / 28672 correct (75.69)
validation_accuracy
Got 1610 / 3584 correct (44.92)

Iteration 200, loss = 0.3008
train_accuracy
Got 21101 / 28672 correct (73.59)
validation_accuracy
Got 1497 / 3584 correct (41.77)

Iteration 300, loss = 0.3244
train_accuracy
Got 20350 / 28672 correct (70.98)
validation_accuracy
Got 1571 / 3584 correct (43.83)

Iteration 400, loss = 0.2193
train_accuracy


Got 1578 / 3584 correct (44.03)

Iteration 100, loss = 0.1059
train_accuracy
Got 25174 / 28672 correct (87.80)
validation_accuracy
Got 1619 / 3584 correct (45.17)

Iteration 200, loss = 0.0486
train_accuracy
Got 24585 / 28672 correct (85.75)
validation_accuracy
Got 1639 / 3584 correct (45.73)

Iteration 300, loss = 0.0936
train_accuracy
Got 25315 / 28672 correct (88.29)
validation_accuracy
Got 1650 / 3584 correct (46.04)

Iteration 400, loss = 0.2267
train_accuracy
Got 24329 / 28672 correct (84.85)
validation_accuracy
Got 1654 / 3584 correct (46.15)

Got 1626 / 3584 correct (45.37)

epoch48
Iteration 0, loss = 0.0945
train_accuracy
Got 24978 / 28672 correct (87.12)
validation_accuracy
Got 1626 / 3584 correct (45.37)

Iteration 100, loss = 0.1385
train_accuracy
Got 24788 / 28672 correct (86.45)
validation_accuracy
Got 1590 / 3584 correct (44.36)

Iteration 200, loss = 0.1445
train_accuracy
Got 24764 / 28672 correct (86.37)
validation_accuracy
Got 1658 / 3584 correct (46.26)

Iteration 3

Got 1701 / 3584 correct (47.46)

epoch59
Iteration 0, loss = 0.0356
train_accuracy
Got 27088 / 28672 correct (94.48)
validation_accuracy
Got 1688 / 3584 correct (47.10)

Iteration 100, loss = 0.0433
train_accuracy
Got 27173 / 28672 correct (94.77)
validation_accuracy
Got 1647 / 3584 correct (45.95)

Iteration 200, loss = 0.0931
train_accuracy
Got 26817 / 28672 correct (93.53)
validation_accuracy
Got 1642 / 3584 correct (45.81)

Iteration 300, loss = 0.0122
train_accuracy
Got 26978 / 28672 correct (94.09)
validation_accuracy
Got 1634 / 3584 correct (45.59)

Iteration 400, loss = 0.0969
train_accuracy
Got 26996 / 28672 correct (94.15)
validation_accuracy
Got 1662 / 3584 correct (46.37)

Got 1656 / 3584 correct (46.21)



In [16]:
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                     momentum=0.9, nesterov=True)

train_part34(model,optimizer,train_data,train_label,public_data,public_label,epochs=10)

epoch0
Iteration 0, loss = 0.0459
train_accuracy
Got 27015 / 28672 correct (94.22)
validation_accuracy
Got 1668 / 3584 correct (46.54)

Iteration 100, loss = 0.0593
train_accuracy
Got 27115 / 28672 correct (94.57)
validation_accuracy
Got 1634 / 3584 correct (45.59)

Iteration 200, loss = 0.0600
train_accuracy
Got 25978 / 28672 correct (90.60)
validation_accuracy
Got 1574 / 3584 correct (43.92)

Iteration 300, loss = 0.0630
train_accuracy
Got 26887 / 28672 correct (93.77)
validation_accuracy
Got 1624 / 3584 correct (45.31)

Iteration 400, loss = 0.0696
train_accuracy
Got 27263 / 28672 correct (95.09)
validation_accuracy
Got 1690 / 3584 correct (47.15)

Got 1677 / 3584 correct (46.79)

epoch1
Iteration 0, loss = 0.0199
train_accuracy
Got 26797 / 28672 correct (93.46)
validation_accuracy
Got 1679 / 3584 correct (46.85)

Iteration 100, loss = 0.0897
train_accuracy
Got 26159 / 28672 correct (91.24)
validation_accuracy
Got 1652 / 3584 correct (46.09)

Iteration 200, loss = 0.0245
train_accur

In [20]:
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                     momentum=0.9, nesterov=True)

train_part34(model,optimizer,train_data,train_label,public_data,public_label,epochs=10)

epoch0
Iteration 0, loss = 0.0025
train_accuracy
Got 27832 / 28672 correct (97.07)
validation_accuracy
Got 1713 / 3584 correct (47.80)

Iteration 100, loss = 0.0392
train_accuracy
Got 28339 / 28672 correct (98.84)
validation_accuracy
Got 1692 / 3584 correct (47.21)

Iteration 200, loss = 0.0042
train_accuracy
Got 28170 / 28672 correct (98.25)
validation_accuracy
Got 1739 / 3584 correct (48.52)

Iteration 300, loss = 0.0180
train_accuracy
Got 28392 / 28672 correct (99.02)
validation_accuracy
Got 1724 / 3584 correct (48.10)

Iteration 400, loss = 0.0734
train_accuracy
Got 28141 / 28672 correct (98.15)
validation_accuracy
Got 1714 / 3584 correct (47.82)

Got 1711 / 3584 correct (47.74)

epoch1
Iteration 0, loss = 0.0039
train_accuracy
Got 27565 / 28672 correct (96.14)
validation_accuracy
Got 1705 / 3584 correct (47.57)

Iteration 100, loss = 0.0310
train_accuracy
Got 28320 / 28672 correct (98.77)
validation_accuracy
Got 1694 / 3584 correct (47.27)

Iteration 200, loss = 0.0032
train_accur

In [19]:
check_accuracy_part34(train_data,train_label, model)
check_accuracy_part34(private_data,private_label, model)

Got 27813 / 28672 correct (97.00)
Got 1713 / 3584 correct (47.80)
